# Working out database loading

As this is now, it will fill in the complete database.
If you really want to reset things, you should delete the SQLite db file first.

This'll do for now.

In [1]:
from spotclient import Client
import models

db = models.Database()
session = models.get_session(create_all=True)

c = Client()
ab = c.allbirds()
latest = ab[0]
print(latest['name'])

JQBX :: Conference of the Birds 2021-03-25


In [17]:
def fill_in_albums(session):
    # find all tracks that don't have albums
    # in the normal flow, we don't have the album IDs, so it's about as easy to find tracks that need them
    # and look them up.
    # don't forget to commit the session yourself...
    track_ids = [t.spotify_id for t in session.query(models.Track).filter(models.Track.album == None)]

    # figure out their albums
    api_tracks = c.tracks(track_ids)
    track_album_ids = dict((t['id'], t['album']['id']) for t in api_tracks)
    album_ids = track_album_ids.values()

    # and get the API data for those albums
    api_albums = c.albums(album_ids)
    api_album_dict = dict((a['id'], a) for a in api_albums)

    # and then create those album objects and attach them
    for k,v in track_album_ids.items():
        track = session.query(models.Track).filter(models.Track.spotify_id == k).first()
        track.album = models.Album.get_or_create(session, v, api_album_dict[v])


In [ ]:
# update the database with the latest material
playlist = db.insert_playlist_from_json(session, latest)
fill_in_albums(session) 
session.commit()

In [21]:
!cat create_and_populate_track_search.sql | sqlite3 birdnest.db

In [15]:
track.album.name

'The Roches'